# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span>

<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Backfill Features to the Feature Store</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/1_backfill_feature_groups.ipynb)


## 🗒️ This notebook is divided into the following sections:
1. Fetch historical data
2. Connect to the Hopsworks feature store
3. Create feature groups and insert them to the feature store

![tutorial-flow](../../images/01_featuregroups.png)

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
#!pip install geopy folium streamlit-folium --q

In [2]:
import datetime
import time
import requests
import json

import pandas as pd
import folium

from functions import *

import warnings
warnings.filterwarnings("ignore")

---

## <span style='color:#ff5f27'> 🌍 Representing the Target cities </span>

In [3]:
with open('target_cities.json') as json_file:
    target_cities = json.load(json_file)

## <span style='color:#ff5f27'> 🌫 Processing Air Quality data</span>

### [🇪🇺 EEA](https://discomap.eea.europa.eu/map/fme/AirQualityExport.htm)
#### EEA means European Environmental Agency

In [4]:
target_cities["EU"]

{'Amsterdam': [52.37, 4.89],
 'Athina': [37.98, 23.73],
 'Berlin': [52.52, 13.39],
 'Gdansk': [54.37, 18.61],
 'Kraków': [50.06, 19.94],
 'London': [51.51, -0.13],
 'Madrid': [40.42, -3.7],
 'Marseille': [43.3, 5.37],
 'Milano': [45.46, 9.19],
 'München': [48.14, 11.58],
 'Napoli': [40.84, 14.25],
 'Paris': [48.85, 2.35],
 'Sevilla': [37.39, -6.0],
 'Stockholm': [59.33, 18.07],
 'Tallinn': [59.44, 24.75],
 'Varna': [43.21, 27.92],
 'Wien': [48.21, 16.37]}

In [5]:
df_eu = pd.read_csv("data/backfill_pm2_5_eu.csv")

In [6]:
df_eu.isna().sum().sum()

0

In [7]:
print("Size of this dataframe:", df_eu.shape)

df_eu.sample(3)

Size of this dataframe: (63548, 3)


,city_name,date,pm2_5
57154,Varna,2015-05-11,11.0
42158,Paris,2015-05-27,13.0
34997,München,2016-05-06,10.0


### [🇺🇸 USEPA](https://aqs.epa.gov/aqsweb/documents/data_api.html#daily)
#### USEPA means United States Environmental Protection Agency
[Manual downloading](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)



In [8]:
target_cities["US"]

{'Albuquerque': [35.08, -106.65],
 'Atlanta': [33.75, -84.39],
 'Chicago': [41.88, -87.62],
 'Columbus': [39.96, -83.0],
 'Dallas': [32.78, -96.8],
 'Denver': [39.74, -104.98],
 'Houston': [29.76, -95.37],
 'Los Angeles': [34.05, -118.24],
 'New York': [40.71, -74.01],
 'Phoenix-Mesa': [33.66, -112.04],
 'Salt Lake City': [40.76, -111.89],
 'San Francisco': [37.78, -122.42],
 'Tampa': [27.95, -82.46]}

In [9]:
df_us = pd.read_csv("data/backfill_pm2_5_us.csv")

In [10]:
df_us.isna().sum().sum()

0

In [11]:
print("Size of this dataframe:", df_us.shape)

df_us.sample(3)

Size of this dataframe: (46037, 3)


,date,city_name,pm2_5
41194,2019-08-21,San Francisco,5.2
6836,2021-09-29,Atlanta,15.8
19361,2019-03-31,Denver,8.3


### <span style="color:#ff5f27;">🏢 Processing special city - `Seattle`</span>
#### We need different stations across the Seattle. 
I downloaded daily `PM2.5` data manually [here](https://www.epa.gov/outdoor-air-quality-data/download-daily-data)

In [12]:
target_cities["Seattle"]

{'Bellevue-SE 12th St': [47.60086, -122.1484],
 'DARRINGTON - FIR ST (Darrington High School)': [48.2469, -121.6031],
 'KENT - JAMES & CENTRAL': [47.38611, -122.23028],
 'LAKE FOREST PARK TOWNE CENTER': [47.755, -122.2806],
 'MARYSVILLE - 7TH AVE (Marysville Junior High)': [48.05432, -122.17153],
 'NORTH BEND - NORTH BEND WAY': [47.49022, -121.77278],
 'SEATTLE - BEACON HILL': [47.56824, -122.30863],
 'SEATTLE - DUWAMISH': [47.55975, -122.33827],
 'SEATTLE - SOUTH PARK #2': [47.53091, -122.3208],
 'Seattle-10th & Weller': [47.59722, -122.31972],
 'TACOMA - ALEXANDER AVE': [47.2656, -122.3858],
 'TACOMA - L STREET': [47.1864, -122.4517],
 'Tacoma-S 36th St': [47.22634, -122.46256],
 'Tukwila Allentown': [47.49854, -122.27839],
 'Tulalip-Totem Beach Rd': [48.06534, -122.28519]}

In [13]:
df_seattle = pd.read_csv("data/backfill_pm2_5_seattle.csv")

In [14]:
df_seattle.isna().sum().sum()

0

In [15]:
print("Size of this dataframe:", df_seattle.shape)

df_seattle.sample(3)

Size of this dataframe: (46479, 3)


,city_name,date,pm2_5
5586,KENT - JAMES & CENTRAL,2014-07-08,4.2
34068,MARYSVILLE - 7TH AVE (Marysville Junior High),2020-01-28,5.8
6776,DARRINGTON - FIR ST (Darrington High School),2014-11-14,16.7


In [16]:
df_seattle.city_name.value_counts()

NORTH BEND - NORTH BEND WAY                      3705
TACOMA - L STREET                                3696
SEATTLE - BEACON HILL                            3691
MARYSVILLE - 7TH AVE (Marysville Junior High)    3648
DARRINGTON - FIR ST (Darrington High School)     3614
SEATTLE - SOUTH PARK #2                          3577
TACOMA - ALEXANDER AVE                           3569
KENT - JAMES & CENTRAL                           3556
SEATTLE - DUWAMISH                               3439
Seattle-10th & Weller                            3097
LAKE FOREST PARK TOWNE CENTER                    2999
Tacoma-S 36th St                                 2574
Bellevue-SE 12th St                              2172
Tukwila Allentown                                2074
Tulalip-Totem Beach Rd                           1068
Name: city_name, dtype: int64

### <span style="color:#ff5f27;">🌟 All together</span>

In [17]:
df_air_quality = pd.concat([df_eu, df_us, df_seattle]).reset_index(drop=True)

In [18]:
df_air_quality.sample(5)

,city_name,date,pm2_5
24437,Madrid,2018-04-04,3.0
147808,TACOMA - L STREET,2021-08-20,3.2
97329,Phoenix-Mesa,2019-05-03,8.7
119482,TACOMA - L STREET,2015-10-07,7.4
49627,Stockholm,2015-04-20,5.0


In [19]:
df_air_quality.shape

(156064, 3)

In [20]:
df_air_quality.columns

Index(['city_name', 'date', 'pm2_5'], dtype='object')

---

## <span style='color:#ff5f27'> 🌦 Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

In [21]:
df_weather = pd.read_csv("data/backfill_weather.csv")

In [22]:
df_weather.city_name.value_counts()

Amsterdam                                        3767
Houston                                          3767
New York                                         3767
Phoenix-Mesa                                     3767
Salt Lake City                                   3767
San Francisco                                    3767
Tampa                                            3767
Bellevue-SE 12th St                              3767
DARRINGTON - FIR ST (Darrington High School)     3767
KENT - JAMES & CENTRAL                           3767
LAKE FOREST PARK TOWNE CENTER                    3767
MARYSVILLE - 7TH AVE (Marysville Junior High)    3767
NORTH BEND - NORTH BEND WAY                      3767
SEATTLE - BEACON HILL                            3767
SEATTLE - DUWAMISH                               3767
SEATTLE - SOUTH PARK #2                          3767
Seattle-10th & Weller                            3767
TACOMA - ALEXANDER AVE                           3767
TACOMA - L STREET           

In [23]:
df_weather.sample(3)

,city_name,date,temperature_max,temperature_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_max,wind_gusts_max,wind_direction_dominant
48110,Sevilla,2021-03-10,19.0,6.9,0.0,0.0,0.0,0.0,9.2,14.8,326
13698,Gdansk,2019-08-16,21.9,14.9,7.5,7.5,0.0,7.0,18.2,28.1,197
147993,Seattle-10th & Weller,2016-09-15,24.3,11.0,0.0,0.0,0.0,0.0,11.5,23.8,197


---

In [24]:
df_air_quality.date = pd.to_datetime(df_air_quality.date)
df_weather.date = pd.to_datetime(df_weather.date)

df_air_quality["unix_time"] = df_air_quality["date"].apply(convert_date_to_unix)
df_weather["unix_time"] = df_weather["date"].apply(convert_date_to_unix)

In [25]:
df_air_quality.date = df_air_quality.date.astype(str)
df_weather.date = df_weather.date.astype(str)

In [26]:
df_air_quality

,city_name,date,pm2_5,unix_time
0,Amsterdam,2013-01-01,14.0,1356998400000
1,Amsterdam,2013-01-02,8.0,1357084800000
2,Amsterdam,2013-01-03,12.0,1357171200000
3,Amsterdam,2013-01-04,12.0,1357257600000
4,Amsterdam,2013-01-05,14.0,1357344000000
...,...,...,...,...
156059,MARYSVILLE - 7TH AVE (Marysville Junior High),2023-03-30,7.9,1680134400000
156060,MARYSVILLE - 7TH AVE (Marysville Junior High),2023-03-31,3.7,1680220800000
156061,MARYSVILLE - 7TH AVE (Marysville Junior High),2023-04-01,3.4,1680307200000
156062,MARYSVILLE - 7TH AVE (Marysville Junior High),2023-04-02,3.1,1680393600000


---

### <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [27]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://35.240.110.235:443/p/119


Connected. Call `.close()` to terminate connection gracefully.


In [28]:
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

expectation_suite = ExpectationSuite(
    expectation_suite_name="transaction_suite")

expectation_suite.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column":"pm2_5", 
            "min_value":"0.0",
            "max_value":"1000.0",
        }
    )
)


{"kwargs": {"column": "pm2_5", "min_value": "0.0", "max_value": "1000.0"}, "expectation_type": "expect_column_values_to_be_between", "meta": {}}

## <span style="color:#ff5f27;">🪄 Creating Feature Groups</span>

### <span style='color:#ff5f27'> 🌫 Air Quality Data

In [29]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city_name'], #'unix_time',
    online_enabled=False,
    expectation_suite = expectation_suite,
    event_time=["unix_time"]
)    

In [30]:
air_quality_fg.insert(df_air_quality, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://35.240.110.235:443/p/119/fs/67/fg/21


Uploading Dataframe: 0.00% |          | Rows 0/156064 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://35.240.110.235/p/119/jobs/named/air_quality_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f89cf1ca3d0>, None)

In [31]:
name = "pm2_5"
f = air_quality_fg.get_feature(name)
air_quality_fg.update_feature_description(name, "particulate matter 2.5 (PM2.5) are tiny particles or droplets in the air that are two and one half microns or less in width")  

### <span style='color:#ff5f27'> 🌦 Weather Data

In [32]:
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city_name'], #'unix_time'
    online_enabled=False,
    event_time=["unix_time"]
) 

In [33]:
weather_fg.insert(df_weather, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://35.240.110.235:443/p/119/fs/67/fg/22


Uploading Dataframe: 0.00% |          | Rows 0/169515 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://35.240.110.235/p/119/jobs/named/weather_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f89cd71abb0>, None)